# Smurf Predictor | Brandon Jackson || Richard Tang || Tuan Quach 
# This ambitious data science experiment's goal is: given an input (summoner name), return a list of possible smurfs of that account. This project was inspried by an interview I watched with Team Liquid's coach Steve mentioned that when teams are scouting players they often need to get a full list of smurf accounts.
****
# For those of you who do not play the game League of Legends, often times a high elo player (the rating system for the game) create smurf accounts to practice new champions or get lower queue time (since they must be matched up with players of their skill level as you climb the elo ladder you have fewer players as good as you). You may also want to hide a pocket pick you may bring out in the LCS/LCK/LPL/LMS/etc...(tournment) So there is insentive for coaches to want to know who other players may be practicing on as well as recruits they may be scouting and want to get a fuller picture. Out of respect for player's who want to hide their smurf some code that would be avalible to public on github will not be posted. However, if you are a coach or team looking for software like this please contact me at Brandon@devbjax.com(Prefered) || Bjax@cs.umd.edu.

# Before we begin, a bit of knowledge of Leaue of Legends. Leauge of Legeneds is a MOBA game, Massive-Online-Battle-Arena, in which 10 players enter the arena and are split among into 2 teams. Each team has 5 roles to fill, Brusier/Tank, Mage, Jungler, DPS, and Support. each role offering different aspect to the game. In order to fill those roles, players pick among 141+ champions which offer a wide variety of options to fill in their roles. The goal of this game is to win as a group/team with synergy with other teammates or with champion's ability.

##We first need to grab information, Best place to grab data from a player is the game company itself, Riot.##

# Steps: 
1.   Head to https://developer.riotgames.com/
2.   Make a account
3.   Grab a API key(keep in mind that it refreshs every 24 hours)
4.   Install the libraries require by Riot to grab the data



In [2]:
!pip install riotwatcher

    100% |████████████████████████████████| 51kB 2.5MB/s 


# Create a list of dataframes continaing games that a summoner has played

## Constraints, when picking a player that a target player, the player being compared must at least have as many games. Obviosuly the more the better, for this instance the target player has ~127 games last (last we checked) and the player we are looking at had much more. We limited it to only 100

In [0]:
from riotwatcher import RiotWatcher
import json
import requests
import pandas as pd
from pandas.io.json import json_normalize
import time

def parse_get(url):
  response = requests.request("GET", url)
  return json.loads(response.text)

def get_player(summonerName, amount_of_games):
  api_key_raw = 'RGAPI-85b29e64-6afd-4bc3-a4c3-c58da98410f3'
  api_key = "?api_key=" + str(api_key_raw)
  #Need to generate a new riot API key every 24 hours
  watcher = RiotWatcher(api_key_raw)

  my_region = 'na1'
  #summonerName = 'Doublelift'
  me = watcher.summoner.by_name(my_region, summonerName)
  #print(me)

  account_id = str(me['accountId'])
  #Get list of match ID's by account
  url_matchlists = "https://na1.api.riotgames.com/lol/match/v3/matchlists/by-account/" + account_id + api_key
  match_list = parse_get(url_matchlists)

  matches = []
  lanes = []
  for i in range(match_list['endIndex']):
    matches.append(match_list['matches'][i]['gameId'])
    lanes.append(match_list['matches'][i]['lane'])
  ''' a full match look like this
  {'champion': 64,
   'gameId': 2915177882,
   'lane': 'JUNGLE',
   'platformId': 'NA1',
   'queue': 420,
   'role': 'NONE',
   'season': 11,
   'timestamp': 1542855765895}
  '''

  data_list = []
  counter = 0
  for i in matches:
    time.sleep(3) #If this doesn't work increase the wait time as Riot's API has rate limits
    counter = counter + 1
    if(counter == amount_of_games + 1):
      break
    #display(i, counter)
    #Get match details from matchID from matchlist
    url_match = "https://na1.api.riotgames.com/lol/match/v4/matches/"+ str(i) + api_key 
    url_timeline = "https://na1.api.riotgames.com/lol/match/v4/timelines/by-match/"+ str(i) + api_key

    #Cross reffrence so that we can figure out who is in timeslines since timeliensv4 does not give summer information
    #However we can use another repsonse to get it using matchesv4
    response = requests.request("GET", url_match)
    parsed_json = json.loads(response.text)
    index = 0
    for j in range(10):
       if summonerName == parsed_json['participantIdentities'][j]['player']['summonerName']:
          index = j
          break

    participantId = parsed_json['participantIdentities'][index]['participantId']
    parsed_json = parse_get(url_timeline)
    #display(parsed_json)
    #Filter dataframe for given summer
    data = json_normalize(parsed_json['frames'])
    list(data)
    filter_list = list(data)
    res = [k for k in filter_list if 'participantFrames.'+str(participantId)+'.' in k]
    res
    data = data[res]
    data_list.append(data)

  return data_list, me

# This is where we start collecting player's information over a certain amount of games. We choose Doublelift as he is a famous pro player among League of Legends. The player's name can change but for now, as a test subject we have input Doublelift.

In [0]:
list_of_matches, player = get_player("Doublelift", 100)

# this part is where you would fill in a random players name in order to compare a famous pro player with a random player in order to determine if this is a smurf account or just some random high end player.

In [0]:
list_of_matches2, player2 = get_player("WildTurtle", 10)

# Note: as a group, we decided it would be quicker to download the data into our local storage instead of constantly grabbing the data from Riot's API 

In [0]:
def download_as_csv(list):
  from google.colab import files
  counter = 0
  for i in list:
    counter += 1
    i.to_csv('game'+str(counter)+'.csv')
    files.download('game'+str(counter)+'.csv')

### Imports and seed

In [7]:
import numpy as np
import keras
from keras.models import Sequential
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline

# fix random seed for reproducibility
seed = 7
np.random.seed(seed)


Using TensorFlow backend.


### Load CSV since riot has rate limits, it's best to save the data so we can call whenever we want WITHOUT having to run get_player which takes not only forever but is prone to error due to rate limits.

In [8]:
from google.colab import files
uploaded = files.upload()

Use this if you are using google colab and would like ot upload a game you downloaded as CSV

In [0]:
#print (uploaded['game1.csv'][:200].decode('utf-8') + '...')

### Reason for using certain features. While some are obvious, other require more domain knowledge. We keep current gold and total gold as tolal gold and level indicate how well a player is doing. Current gold however indicates how many times a player returns to base to buy items during a game which is highly detemined by a player philosophy and knowledge about the game.

### We want to flatten each dataframe so that we can have one observation be a row instead of being an entire CSV file. This is quite crude but will suffice for this tutorial. We are testing our hypothesis that it is possible to tell players apart by the data given to us by riot games then standardizing it and feeding it to a model. A game is a collection fo frames via riot. We don't know what a frame is however, it is th eunit that riot uses when you get a callback from their API. So we use frames because the amount of frames changes as the games go on, so we can average out each game with accurcy by using frames.

In [0]:
# import pandas as pd
import io

# df = pd.read_csv(io.StringIO(uploaded['game1.csv'].decode('utf-8')))
# df
def collapse_into_df(list, player_id):
  #collapse_df = pd.DataFrame(columns=['currentGold', 'jungleMinionsKilled', 'level', 'minionsKilled','position_x', 'position_y','totalGold', 'xp'])
  #print(df.columns)
  rows_list = []
  for df in list:
    #Will need to add 'Unnamed: 0' to the column assigment below if reading from CSV
    df.columns = ['currentGold', 'dominionScore', 'jungleMinionsKilled', 'level', 'minionsKilled', 'participantId','position_x', 'position_y', 'teamScore','totalGold', 'xp']
    df = df[['currentGold', 'jungleMinionsKilled', 'level', 'minionsKilled','position_x', 'position_y','totalGold', 'xp']]
    dict = {}
    current_Gold_avg = 0
    #Remember these are per frame
    for k in df["currentGold"]:
      current_Gold_avg += k/len(df.index)
    dict['currentGold_per_frame'] = current_Gold_avg
    dict['totalGold_per_frame'] = (df["currentGold"][len(df.index)-1])/(len(df.index)-1)
    dict['jungleMinionsKilled_per_frame'] = (df["jungleMinionsKilled"][len(df.index)-1])/(len(df.index)-1)
    dict['level_per_frame'] = (df["level"][len(df.index)-1])/(len(df.index)-1)
    dict['minionsKilled_per_frame'] = (df["minionsKilled"][len(df.index)-1])/(len(df.index)-1)
    dict['totalGold_per_frame'] = (df["totalGold"][len(df.index)-1])/(len(df.index)-1)
    dict['xp_per_frame'] = (df["xp"][len(df.index)-1])/(len(df.index)-1)
    dict['player'] = player_id
    rows_list.append(dict)
  collapse_df = df = pd.DataFrame(rows_list)
  return collapse_df

### Collaspe the list of games we compiled above into a single dataframe. Each row will be representive of a game and we can get an idea of what the player is like between games on average. We hope this will give us a picture of how to determine the player's play style.

In [11]:
data = collapse_into_df(list_of_matches, player['accountId']) #The list of Doublelift's games and his account_id
data

,currentGold_per_frame,jungleMinionsKilled_per_frame,level_per_frame,minionsKilled_per_frame,player,totalGold_per_frame,xp_per_frame
0,627.333333,4.846154,0.500000,0.846154,32971449,360.076923,423.000000
1,684.200000,0.666667,0.541667,5.875000,32971449,408.208333,440.958333
2,1066.400000,0.758621,0.517241,8.000000,32971449,503.448276,473.068966
3,876.217391,0.818182,0.636364,8.136364,32971449,650.590909,549.863636
4,1072.735294,1.333333,0.515152,8.333333,32971449,500.181818,535.030303
5,1066.045455,0.558140,0.418605,7.279070,32971449,526.697674,538.744186
6,498.181818,0.000000,0.468750,2.437500,32971449,316.750000,429.593750
7,525.153846,0.160000,0.520000,8.480000,32971449,371.560000,421.160000
8,1216.454545,0.930233,0.418605,8.348837,32971449,528.627907,529.883721
9,495.925926,0.307692,0.500000,6.692308,32971449,390.576923,400.269231


### Same thing for the player we want to check againt to see if they are a smurf.

In [12]:
data2 = collapse_into_df(list_of_matches2, player2['accountId']) #The list of Doublelift's games and his account_id
data2

,currentGold_per_frame,jungleMinionsKilled_per_frame,level_per_frame,minionsKilled_per_frame,player,totalGold_per_frame,xp_per_frame
0,473.892857,0.000000,0.518519,6.777778,548787,343.592593,431.925926
1,599.851852,0.461538,0.538462,7.307692,548787,405.692308,451.269231
2,722.966667,0.517241,0.482759,7.551724,548787,399.689655,423.517241
3,487.888889,0.514286,0.514286,6.714286,548787,558.771429,564.285714
4,879.218750,0.935484,0.548387,7.354839,548787,634.548387,587.419355
5,537.111111,0.153846,0.615385,7.307692,548787,446.461538,599.269231
6,670.944444,0.235294,0.588235,7.823529,548787,514.058824,384.941176
7,792.777778,0.685714,0.485714,6.114286,548787,409.428571,493.457143
8,566.150000,0.210526,0.631579,7.368421,548787,449.421053,459.947368
9,690.555556,0.384615,0.538462,6.730769,548787,432.769231,474.807692


### Get an idea of what the data looks like

In [13]:
df = data.append(data2, ignore_index=True)
df.describe()

,currentGold_per_frame,jungleMinionsKilled_per_frame,level_per_frame,minionsKilled_per_frame,player,totalGold_per_frame,xp_per_frame
count,110.000000,110.000000,110.000000,110.000000,1.100000e+02,110.000000,110.000000
mean,781.349642,0.828938,0.620706,6.753268,3.002393e+07,523.195619,561.551594
std,276.267767,1.318309,0.212207,2.527327,9.363519e+06,208.037753,242.922659
min,371.823529,0.000000,0.418605,0.266667,5.487870e+05,272.041667,281.250000
25%,581.241071,0.181818,0.500000,6.377083,3.297145e+07,391.380435,424.115204
50%,705.292637,0.414230,0.541667,7.490741,3.297145e+07,450.418860,475.750000
75%,896.105769,0.747685,0.590241,8.457386,3.297145e+07,565.515584,552.814394
max,1791.937500,6.133333,1.230769,10.542857,3.297145e+07,1297.692308,1408.125000


### Uncomment the line below if you wanna see what your data looks like. Warning, sometimes the .hist() will overlay

In [0]:
from matplotlib.pyplot import figure
import matplotlib.pyplot as plt
#df.hist()

# Create the model
### This is the important and most confusing part of the project. If you are new to data science or machine learning this will seem very foreign. We will use a neural network from Keras to create a model which will use binary cross entropy. basically what we are doing is learning how the target player (Doublelift) plays the game. Our goal is to get a good picture of who he is. Then we want to get our test player (Wildturtle) and see if his games are somewhat similar or not.

### Quick crash course on neural nets. We model the brain (or at least we use to) in an attempt to learn like humans do. However, we have layers, nodes, and activation functions for neurons and synapses

In [0]:
class MyLogger(keras.callbacks.Callback):
  def __init__(self, n):
    self.n = n   # print loss & acc every n epochs

  def on_epoch_end(self, epoch, logs={}):
    if epoch % self.n == 0:
      curr_loss =logs.get('loss')
      curr_acc = logs.get('acc') * 100
      print("epoch = %4d loss = %0.6f acc = %0.2f%%" % \
        (epoch, curr_loss, curr_acc))

In [0]:
msk = np.random.rand(len(data)) < 0.8
train = data[msk]
test = data[~msk]

from sklearn.model_selection import train_test_split
train, test = train_test_split(df, test_size=0.2)
train_x = train[['currentGold_per_frame',	'jungleMinionsKilled_per_frame',	'level_per_frame',	'minionsKilled_per_frame', 'totalGold_per_frame',	'xp_per_frame']]
train_y = train[['player']]

test_x = test[['currentGold_per_frame',	'jungleMinionsKilled_per_frame',	'level_per_frame',	'minionsKilled_per_frame', 'totalGold_per_frame',	'xp_per_frame']]
test_y = test[['player']]

In [0]:
my_init = keras.initializers.glorot_uniform(seed=1)
model = keras.models.Sequential()
# model.add(keras.layers.Dense(units=8, input_dim=6,
#   activation='relu', kernel_initializer=my_init)) 
# model.add(keras.layers.Dense(units=8, activation='relu',
#   kernel_initializer=my_init)) 
# model.add(keras.layers.Dense(units=1, activation='sigmoid',
#   kernel_initializer=my_init))
model.add(Dense(6, input_dim=6, kernel_initializer='normal', activation='relu'))
model.add(Dense(1, kernel_initializer='normal', activation='sigmoid'))
simple_sgd = keras.optimizers.SGD(lr=0.01)  
model.compile(loss='binary_crossentropy',
  optimizer=simple_sgd, metrics=['accuracy']) 

In [0]:
max_epochs = 500
my_logger = MyLogger(n=50)
h = model.fit(train_x, train_y, batch_size=32,
  epochs=max_epochs, verbose=0)
mp = ".\\Models\\banknote_model.h5"
model.save(mp)

In [0]:
np.set_printoptions(precision=4, suppress=True)
eval_results = model.evaluate(test_x, test_y, verbose=0) 

In [25]:
inputs = data2[['currentGold_per_frame', 'jungleMinionsKilled_per_frame',  'level_per_frame',  'minionsKilled_per_frame', 'totalGold_per_frame', 'xp_per_frame']]
#inpts = np.array([[0.5, 0.5, 0.5, 0.5, 0.5, 0.5]], dtype=np.float32)
pred = model.predict(inputs.iloc[[0]])
print("\nPredicting authenticity for: ")
print(inputs.iloc[[0]])
print("Probability that class = 1 (fake):")
print(pred)


Predicting authenticity for: 
   currentGold_per_frame  jungleMinionsKilled_per_frame  level_per_frame  \
0             473.892857                            0.0         0.518519   

   minionsKilled_per_frame  totalGold_per_frame  xp_per_frame  
0                 6.777778           343.592593    431.925926  
Probability that class = 1 (fake):
[[1.]]


### We can see we got a 1 which means Wildturtle is not Doublelift!

****
# In Conclusion,

Looking at the data we can conclude that WildTurtle and Doublelift are two different people. This project goal is to determine if these two accounts are the same player. The project allows other various uses beside determing players such as gathering data from other professional teams and offering counterplay towards or 
using the data for trainning.

# Helpful Links


*   [Further idea] https://medium.com/pandascore-stories/league-of-legends-getting-champion-coordinates-from-the-minimap-using-deep-learning-48a49d35bb74
*   [Riot API] https://developer.riotgames.com/
*   [Locating Player account] http://na.op.gg/
*   [Pandas Tutorial] https://pandas.pydata.org/pandas-docs/stable/tutorials.html
*   [Offical League of Legends] https://play.na.leagueoflegends.com/en_US
*   [Offical Esports of League Of Legends]https://www.lolesports.com/en_US







